In [1]:
from avalanche.benchmarks.classic import SplitCIFAR100
from tqdm import tqdm
import torch

/home/giacomo/mambaforge/envs/continual/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Dataset

In [2]:
split_cifar100 = SplitCIFAR100(n_experiences=10, seed=1)

Files already downloaded and verified
Files already downloaded and verified


In [3]:
# Original train/test sets
print('--- Original datasets:')
print(split_cifar100.original_train_dataset)
print(split_cifar100.original_test_dataset)

# A list describing which training patterns are assigned to each experience.
# Patterns are identified by their id w.r.t. the dataset found in the
# original_train_dataset field.
print('--- Train patterns assignment:')
print(split_cifar100.train_exps_patterns_assignment)

# A list describing which test patterns are assigned to each experience.
# Patterns are identified by their id w.r.t. the dataset found in the
# original_test_dataset field
print('--- Test patterns assignment:')
print(split_cifar100.test_exps_patterns_assignment)

# the task label of each experience.
print('--- Task labels:')
print(split_cifar100.task_labels)

# train and test streams
print('--- Streams:')
print(split_cifar100.train_stream)
print(split_cifar100.test_stream)

# A list that, for each experience (identified by its index/ID),
# stores a set of the (optionally remapped) IDs of classes of patterns
# assigned to that experience.
print('--- Classes in each experience:')
print(split_cifar100.original_classes_in_exp)

--- Original datasets:
--- Train patterns assignment:
[[6, 11, 13, 31, 41, 47, 62, 65, 72, 91, 92, 121, 131, 142, 143, 147, 157, 158, 163, 171, 175, 184, 185, 187, 205, 213, 230, 231, 232, 236, 245, 276, 296, 347, 349, 363, 366, 368, 376, 418, 435, 447, 452, 455, 486, 501, 504, 506, 522, 537, 540, 546, 549, 552, 556, 579, 584, 595, 597, 640, 642, 691, 695, 700, 704, 726, 734, 745, 746, 758, 771, 782, 790, 791, 797, 816, 817, 824, 826, 830, 846, 852, 863, 867, 876, 879, 882, 890, 891, 893, 904, 927, 941, 955, 972, 999, 1003, 1008, 1010, 1034, 1044, 1049, 1050, 1067, 1074, 1095, 1100, 1108, 1109, 1115, 1116, 1141, 1145, 1150, 1169, 1171, 1172, 1181, 1182, 1187, 1199, 1202, 1218, 1222, 1250, 1254, 1266, 1271, 1277, 1284, 1303, 1317, 1339, 1347, 1381, 1399, 1401, 1419, 1424, 1433, 1440, 1443, 1449, 1452, 1455, 1476, 1492, 1497, 1502, 1543, 1553, 1560, 1573, 1586, 1588, 1595, 1603, 1607, 1614, 1617, 1648, 1649, 1667, 1676, 1678, 1685, 1686, 1689, 1693, 1700, 1709, 1710, 1724, 1727, 1732, 17

In [4]:
# we get the first experience
experience = split_cifar100.train_stream[1]

# task label and dataset are the main attributes
t_label = experience.task_label
dataset = experience.dataset
print("\nNumber of examples:", len(dataset))
print("Task Label:", t_label)

# but you can recover additional info
print('--- Additional info:')
print('Experience ID:', experience.current_experience)
print('Classes in this experience:', experience.classes_in_this_experience)
print('Classes seen so far:', experience.classes_seen_so_far)
print('Previous classes:', experience.previous_classes)
print('Future classes:', experience.future_classes)
print('Origin stream:', experience.origin_stream)
print('Benchmark:', experience.benchmark)


Number of examples: 5000
Task Label: 0
--- Additional info:
Experience ID: 1
Classes in this experience: [3, 36, 11, 80, 48, 16, 22, 23, 58, 62]
Classes seen so far: [3, 72, 11, 15, 80, 16, 79, 17, 22, 23, 90, 96, 32, 35, 36, 45, 48, 58, 62, 63]
Previous classes: [96, 32, 35, 72, 45, 15, 79, 17, 90, 63]
Future classes: [0, 1, 2, 4, 5, 6, 7, 8, 9, 10, 12, 13, 14, 18, 19, 20, 21, 24, 25, 26, 27, 28, 29, 30, 31, 33, 34, 37, 38, 39, 40, 41, 42, 43, 44, 46, 47, 49, 50, 51, 52, 53, 54, 55, 56, 57, 59, 60, 61, 64, 65, 66, 67, 68, 69, 70, 71, 73, 74, 75, 76, 77, 78, 81, 82, 83, 84, 85, 86, 87, 88, 89, 91, 92, 93, 94, 95, 97, 98, 99]
Origin stream: <avalanche.benchmarks.scenarios.classification_scenario.ClassificationStream object at 0x7fca9599b280>
Benchmark: <avalanche.benchmarks.scenarios.new_classes.nc_scenario.NCScenario object at 0x7fcbfcbbc850>


# Split training into train and validation

In [5]:
from avalanche.benchmarks.generators import benchmark_with_validation_stream, class_balanced_split_strategy

Split the training set into a training and validation set

In [6]:
validation_size = 0.2
class_balanced_split = lambda exp: class_balanced_split_strategy(validation_size, exp)
split_cifar100_with_valid = benchmark_with_validation_stream(split_cifar100, custom_split_strategy=class_balanced_split, )

In [7]:
print(len(split_cifar100.train_stream[0].dataset))
print(len(split_cifar100.test_stream[0].dataset))

5000
1000


In [8]:
print(len(split_cifar100_with_valid.train_stream[0].dataset))
print(len(split_cifar100_with_valid.valid_stream[0].dataset))
print(len(split_cifar100_with_valid.test_stream[0].dataset))

4000
1000
1000


# Train

In [9]:
from torchvision.models import resnet18
from torch.nn import CrossEntropyLoss
from avalanche.training import Naive, Replay

In [10]:
if torch.cuda.is_available():       
        device = torch.device("cuda")
        print(f'There are {torch.cuda.device_count()} GPU(s) available.')
        print('Device name:', torch.cuda.get_device_name(0))

else:
    print('No GPU available, using the CPU instead.')
    device = torch.device("cpu")

There are 1 GPU(s) available.
Device name: NVIDIA GeForce RTX 3080 Ti Laptop GPU


In [11]:
from avalanche.training.plugins import EvaluationPlugin
from avalanche.evaluation.metrics import forgetting_metrics, accuracy_metrics, loss_metrics, confusion_matrix_metrics
from avalanche.logging import InteractiveLogger, TextLogger, TensorboardLogger

In [12]:
eval_plugin = EvaluationPlugin(
    accuracy_metrics(minibatch=True, epoch=True, experience=True, stream=True),
    loss_metrics(minibatch=True, epoch=True, experience=True, stream=True),
    forgetting_metrics(experience=True, stream=True),
    loggers=[InteractiveLogger()],
    strict_checks=False
)

In [13]:
from der import DerPlugin

In [14]:
model = resnet18(weights=None, num_classes=100)
optimizer = torch.optim.SGD(model.parameters(), lr=0.001, momentum=0.9)
criterion = CrossEntropyLoss()
    
cl_strategy = Naive(model, optimizer, criterion, train_mb_size=32, train_epochs=20, eval_mb_size=32,
                    eval_every=-1, device=device, evaluator=eval_plugin, plugins=[DerPlugin(mem_size=500)])

In [15]:
# TRAINING LOOP WITH VALIDATION AT EACH EPOCH
print('Starting experiment...')
results = []
for experience in split_cifar100_with_valid.train_stream:
    print("Start of experience: ", experience.current_experience)
    print("Current Classes: ", experience.classes_in_this_experience)

    cl_strategy.train(experience, [split_cifar100_with_valid.valid_stream])
    print('Training completed')

Starting experiment...
Start of experience:  0
Current Classes:  [96, 32, 35, 72, 45, 15, 79, 17, 90, 63]
-- >> Start of training phase << --
100%|██████████| 125/125 [00:02<00:00, 47.11it/s]
Epoch 0 ended.
	Loss_Epoch/train_phase/train_stream/Task000 = 2.3872
	Loss_MB/train_phase/train_stream/Task000 = 2.0960
	Top1_Acc_Epoch/train_phase/train_stream/Task000 = 0.2362
	Top1_Acc_MB/train_phase/train_stream/Task000 = 0.3750
100%|██████████| 125/125 [00:01<00:00, 81.28it/s]
Epoch 1 ended.
	Loss_Epoch/train_phase/train_stream/Task000 = 1.8792
	Loss_MB/train_phase/train_stream/Task000 = 1.5754
	Top1_Acc_Epoch/train_phase/train_stream/Task000 = 0.3357
	Top1_Acc_MB/train_phase/train_stream/Task000 = 0.2812
100%|██████████| 125/125 [00:01<00:00, 80.96it/s]
Epoch 2 ended.
	Loss_Epoch/train_phase/train_stream/Task000 = 1.7410
	Loss_MB/train_phase/train_stream/Task000 = 1.8383
	Top1_Acc_Epoch/train_phase/train_stream/Task000 = 0.3820
	Top1_Acc_MB/train_phase/train_stream/Task000 = 0.4062
100%|████